# Práctica NLP
Hecha por Rubén Cerezo Cuesta
En esta práctica, vamos a hacer dos modelos de NLP a partir de un dataset de Amazon formado por reseñas de videojuegos. Con esto, esperamos conseguir un análisis de sentimiento. Tras entrenar estos modelos, podremos conseguir una herramienta que catalogue reviews en función de cómo de positivas son. 
Este trabajo está dividido en 4 fases:
- Exploración de datos
- Preprocesamiento de datos
- **División Train/Test y entrenamiento**
- Métricas y conclusiones
En este caso, el corpus elegido, de Amazon, incluye reviews de videojuegos, simplemente por ser un tema que conozco y que he pensado que podría resultar fácil a la hora de reconocer el producto con el que trabajo. 

# Aclaración:
En este caso, he creado dos documentos: 
- División_y_entrenamiento0-1.ipynb
- División_y_entrenamiento1-5.ipynb
Esto se debe a que, he creado dos veces los mismos modelos usando distintos datasets (en uno analizaremos un corpus en el que la variable objetivo tiene valores entre 1 y 5, mientras que en el otro, los valores serán 0-1)

Es de esperar que, en los modelos creados sobre 1-5, podamos conseguir una información más detallada, pero también la accuracy será menor, dado 

#  Modelo 1: Machine Learning (TF-IDF + Linear Regression)

En esta parte de nuestra práctica, realizamos el pipeline completo de entrenamiento de un modelo de Machine Learning para clasificar reviews en función de su puntuación. El flujo seguido es el siguiente:

División del dataset en un conjunto de entrenamiento (75%) y otro de prueba (25%), asegurando aleatoriedad y reproducibilidad mediante una semilla fija.

Creación y ajuste de un vectorizador TF-IDF utilizando únicamente los datos de entrenamiento. Este vectorizador convierte el texto de las reviews en representaciones numéricas, incluyendo uni-, bi- y trigramas, y filtrando términos demasiado frecuentes o demasiado raros.

Transformación del texto de los conjuntos de entrenamiento y prueba mediante el vectorizador ajustado.

Entrenamiento de varios modelos de Regresión Logística probando diferentes valores del hiperparámetro C, que controla el nivel de regularización. Para cada modelo se calcula el accuracy tanto en train como en test.

Selección del mejor modelo basándonos en el mayor accuracy obtenido sobre el conjunto de prueba.

Entrenamiento final del modelo óptimo utilizando el mejor valor de C encontrado.

Guardado del modelo entrenado en disco mediante joblib, permitiendo reutilizarlo posteriormente sin necesidad de volver a entrenarlo.

In [14]:
#Cargamos el archivo .csv 

import pandas as pd

path = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced_preprocessed.csv'

df = pd.read_csv(path)
df.head()

,reviewText,overall
0,"['ps1', 'considered', 'best', 's', 'created', ...",2
1,"['yeah', 'ok', 'easy', 'use', 'controls', 'ol'...",3
2,"['star', 'wars', 'force', 'unleashed', 'launch...",3
3,"['review', 'contain', 'minor', 'spoilers', 'wr...",2
4,"['i', 'm', 'having', 'problems', 'getting', 'd...",1


## División en train y test:
Como podemos ver, la carga de los datos, preprocesado y tokenización ha ido bien, desde un segundo archivo que creamos en el notebook anterior para no perder la información. 
en este caso, para nuestra división, tomaremos la columna reviewText como variable predictora, y overall como variable objetivo. 

In [15]:
#Separamos en train y test con una muestra de train del 75% y una muestra de test del 25% 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['reviewText'],
    df['overall'],
    train_size=0.75,
    test_size=0.25,
    random_state=42,
    shuffle=True
)

In [16]:
X_train.iloc[:10]

4884    ['that', 's', 'folks', 'filed', 'suit', 'jakks...
3163    ['i', 'm', 'going', 'write', 'assuming', 'fami...
490     ['loaded', 'software', 'did', 'nt', 'work', 'c...
862     ['played', 'episode', 'ones', 'missions', 'mis...
1740    ['waited', 'years', 'grid', '2', 'grid', '1', ...
4099    ['love', 'game', 'fantastic', 'missed', 'lots'...
4385    ['overall', 'game', 'high', 'standard', '1', '...
2447    ['disappointed', 'midway', 'stateoftheart', 'p...
4230    ['super', 'mario', '3d', 'land', 'honestly', '...
506     ['wipeout', 'fun', 'little', 'watch', 'talks',...
Name: reviewText, dtype: object

In [17]:
y_train.iloc[:10]

4884    1
3163    4
490     1
862     3
1740    2
4099    5
4385    3
2447    1
4230    3
506     2
Name: overall, dtype: int64

## Extracción de features:

Como vemos, nuestra división en train y test ha sido exitosa, por lo que ahora tenemos que ir a una siguiente parte, que sería la extracción de features: 
En este caso, los hiperparámetros son:
max_df y min_df, que cogen tokens con una frecuencia muy baja o alta y los eliminan como outliers.
max_features: 2500 es un tamaño grande que nos permite contemplar los n-grams más representativos

Antes de nada, hacemos un barrido con el que aseguramos encontrar los mejores parámetros para este 

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(strip_accents='ascii')),
    ('clf', LogisticRegression(max_iter=300))
])

param_grid = {
    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
    'tfidf__min_df': [1, 3, 5],
    'tfidf__max_df': [0.9, 0.95, 0.98],
    'tfidf__max_features': [5000, 10000, 20000],
    'clf__C': [0.1, 1, 3, 10],
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)
grid.fit(X_train, y_train)

print("Mejores parámetros:")
print(grid.best_params_)
print("Mejor score:", grid.best_score_)

Fitting 3 folds for each of 324 candidates, totalling 972 fits
Mejores parámetros:
{'clf__C': 1, 'tfidf__max_df': 0.9, 'tfidf__max_features': 20000, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 3)}
Mejor score: 0.42026666666666673


In [19]:

from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(
    max_df=0.95,
    min_df=3,
    max_features=10000,
    strip_accents='ascii',
    ngram_range=(1, 3)
)
cv.fit(X_train)

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,'ascii'
,lowercase,True
,preprocessor,None
,tokenizer,None
,analyzer,'word'
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [20]:
#Mostramos las primeras 20 entradas del vocabulario aprendido
print(list(cv.vocabulary_.items())[:20])

[('that', 8787), ('folks', 2957), ('suit', 8569), ('end', 2448), ('game', 3130), ('wo', 9753), ('nt', 6014), ('come', 1482), ('future', 3119), ('releases', 7456), ('smackdown', 8134), ('bad', 659), ('wanted', 9531), ('play', 6722), ('half', 4151), ('roster', 7692), ('years', 9900), ('version', 9401), ('missing', 5665), ('key', 4732)]


In [21]:
#  Mostramos el tamaño del vocabulario aprendido
print(len(cv.vocabulary_))

10000


In [22]:
# Transformamos los datos de train y test
X_train_ = cv.transform(X_train)
X_test_ = cv.transform(X_test)

## TF_IDF
Es un indicador de cómo de importantes son estas palabras. mide cuántas veces aparece la palabra en la review, y lo compara con cuántas veces aparece en todo el corpus para darle una puntuación. 

In [23]:
#visualizamos una muestra aleatoria de los datos de entrenamiento
import random

i = random.randint(0, len(X_train))
print('ID: {}'.format(i))
print('Sentiment: {}'.format(y_train.iloc[i]))
print('Review: {}'.format(X_train.iloc[i]))

ID: 564
Sentiment: 1
Review: ['case', 'arrived', 'like', 'picture', 'shown', 'graphic', 'makes', 'case', 'look', 'like', 'belongs', 'teenager', 'construction', 'quality', 'poor', 'strong', 'chemical', 'smell', 'case', 'attempting', 'return', 'itemupdatei', 'returned', 'case', 'instead', 'picked', 'hori', 'ps', 'vita', 'hard', 'case', 'case', '5', 'better', 'quality', 'case']


In [24]:
# Mostramos las palabras con mayor y menor TF_IDF en la reseña seleccionada
doc_vector = X_train_[i]
df_tfidf = pd.DataFrame(doc_vector.T.todense(), index=cv.get_feature_names_out(), columns=['tfidf'])
df_tfidf = df_tfidf[df_tfidf['tfidf'] > 0]

top_n = 10
print('Top {} palabras con el mayor TF_IDF en la reseña {}:\n{}'.format(top_n, i, df_tfidf.sort_values(by=["tfidf"],ascending=False)[:top_n]))
print('\nTop {} palabras con el menor TF_IDF en la reseña {}:\n{}'.format(top_n, i, df_tfidf.sort_values(by=["tfidf"],ascending=False)[-top_n:]))

Top 10 palabras con el mayor TF_IDF en la reseña 564:
                 tfidf
case          0.754038
quality       0.212977
ps            0.194576
teenager      0.190543
construction  0.172411
attempting    0.164422
arrived       0.159424
vita          0.154278
returned      0.153360
shown         0.150785

Top 10 palabras con el menor TF_IDF en la reseña 564:
              tfidf
picked     0.129937
return     0.123159
look like  0.122876
poor       0.116871
instead    0.090962
like       0.090938
makes      0.083374
hard       0.082101
look       0.079944
better     0.066418


Como vemos, aplicar puntuaciones de TF_IDF ha salido bastante bien. 
Las palabras que vemos que tienen un mayor índice en la reseña 1163 como vemos están relacionadas con el sector de la conducción, lo cual nos hace pensar que esta review está relacionada con un videojuego de conducción (habla de camiones, por lo que podría ser Eurotruck Simulator?)
Sin embargo, las palabras con un TF-IDF bajo son aquellas que están asociadas con el sector de los videojuegos, como "graphics", "level", "games" o simplemente son muy utilizadas en el inglés, como "looking", "getting" 

In [25]:

# Entrenamiento y evaluación del modelo de Regresión Logística con diferentes valores de C
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

c_params = [0.01, 0.05, 0.25, 0.5, 1, 10, 100, 1000, 10000]

train_acc = list()
test_acc = list()
for c in c_params:
    lr = LogisticRegression(C=c, solver='lbfgs', max_iter=500)
    lr.fit(X_train_, y_train)

    train_predict = lr.predict(X_train_)
    test_predict = lr.predict(X_test_)

    print ("Accuracy for C={}: {}".format(c, accuracy_score(y_test, test_predict)))

    train_acc.append(accuracy_score(y_train, train_predict))
    test_acc.append(accuracy_score(y_test, test_predict))

# Guardamos el mejor modelo entrenado
import joblib
best_c = c_params[test_acc.index(max(test_acc))]
best_lr = LogisticRegression(C=best_c, solver='lbfgs', max_iter=500)
best_lr.fit(X_train_, y_train)
joblib.dump(best_lr, 'logistic_regression_model.pkl')


Accuracy for C=0.01: 0.2912
Accuracy for C=0.05: 0.3912
Accuracy for C=0.25: 0.4176
Accuracy for C=0.5: 0.4248
Accuracy for C=1: 0.4192
Accuracy for C=10: 0.388
Accuracy for C=100: 0.3832
Accuracy for C=1000: 0.392
Accuracy for C=10000: 0.396


['logistic_regression_model.pkl']

In [26]:
# Como parte de las buenas prácticas de Machine Learning, realizamos de nuevo todo el proceso en una sóla celda, eliminando las partes del código que hemos usado para comprobar que lo estábamos haciendo bien.
#  Esto nos da más control sobre el código,
#Cargamos el archivo .csv 
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
# Cargamos los datos
path = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced_preprocessed.csv'
df = pd.read_csv(path)

X_train, X_test, y_train, y_test = train_test_split(
    df['reviewText'],
    df['overall'],
    train_size=0.75,
    test_size=0.25,
    random_state=42,
    shuffle=True
)

#  pipeline y búsqueda de hiperparámetros
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(strip_accents='ascii')),
    ('clf', LogisticRegression(max_iter=1000))
])

param_grid = {
    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
    'tfidf__min_df': [1, 3, 5],
    'tfidf__max_df': [0.9, 0.95, 0.98],
    'tfidf__max_features': [5000, 10000, 20000],
    'clf__C': [0.1, 1, 3, 10],
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring='accuracy',  
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Mejores parámetros encontrados por GridSearch:")
print(grid.best_params_)
print("Mejor score CV:", grid.best_score_)

# --- usar el mejor pipeline para predecir en test ---
best_pipeline = grid.best_estimator_
y_pred_test = best_pipeline.predict(X_test)

print("Accuracy (test):", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_test))
# Guardamos el mejor modelo entrenado en una carpeta especifica para este modelo 
best_c = c_params[test_acc.index(max(test_acc))]
best_lr = LogisticRegression(C=best_c, solver='lbfgs', max_iter=2000, n_jobs=-1)
best_lr.fit(X_train_, y_train)
model_dir = r"C:\Users\rammu\Documents\projects\NLP_04\práctica\modelos\Logistic_regression_tf_idf0-5"
# Guardar modelo
joblib.dump(best_lr, fr"{model_dir}\model.pkl")
# Guardar vectorizador
joblib.dump(cv, fr"{model_dir}\vectorizer.pkl")

# Guardar X_test y y_test
joblib.dump((X_test, y_test), fr"{model_dir}\test_data.pkl")

Fitting 3 folds for each of 324 candidates, totalling 972 fits
Mejores parámetros encontrados por GridSearch:
{'clf__C': 1, 'tfidf__max_df': 0.9, 'tfidf__max_features': 20000, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 3)}
Mejor score CV: 0.42026666666666673
Accuracy (test): 0.4072
              precision    recall  f1-score   support

           1       0.45      0.58      0.51       229
           2       0.35      0.31      0.33       255
           3       0.31      0.32      0.32       246
           4       0.41      0.31      0.35       263
           5       0.49      0.53      0.51       257

    accuracy                           0.41      1250
   macro avg       0.40      0.41      0.40      1250
weighted avg       0.40      0.41      0.40      1250

Matriz de confusión:
 [[133  52  19  10  15]
 [ 83  79  54  13  26]
 [ 31  52  79  46  38]
 [ 21  26  71  82  63]
 [ 26  17  28  50 136]]


['C:\\Users\\rammu\\Documents\\projects\\NLP_04\\práctica\\modelos\\Logistic_regression_tf_idf0-5\\test_data.pkl']

En este caso, los mejores hiperparámetros, que hemos elegido, han sido comprobados tras hacer un barrido que iba probando todas las posibles combinaciones de manera que, sin duda, hemos elegido la mejor posible 

# Modelo nº 2: Machine Learning (Bag of Words + Regresión Logística)

En esta parte de la práctica realizamos el proceso completo de preparación, vectorización y entrenamiento de un modelo de Machine Learning para clasificar reviews según su puntuación. El flujo seguido es el siguiente
- Carga del dataset
Se carga el archivo CSV preprocesado que contiene las reseñas y sus puntuaciones. También se visualizan las primeras filas para comprobar que los datos se han leído correctamente.

- Creación y ajuste del vectorizador Bag of Words (CountVectorizer)
Se construye un vectorizador basado en el modelo Bag of Words, que transforma cada review en una matriz dispersa donde cada columna representa una palabra del vocabulario y cada fila corresponde a un documento.
Se ajusta el vectorizador a todas las reviews.
Se extraen las palabras del vocabulario usando get_feature_names_out().
Opcionalmente, se convierte la matriz dispersa en un DataFrame para visualizar la representación numérica de los textos.

- División del dataset en entrenamiento y prueba
Se divide la matriz generada (X) y las etiquetas (y) en:
75% → entrenamiento
25% → prueba
Esta división se realiza de forma aleatoria pero reproducible, gracias a la semilla random_state=42.

- Entrenamiento del modelo de Regresión Logística
Se entrena un modelo de Regresión Logística utilizando las representaciones Bag of Words del conjunto de entrenamiento.
Se ajusta el modelo con max_iter=500 para asegurar la convergencia.
El modelo aprende a asociar patrones de palabras con las puntuaciones de las reviews.

- Guardado del modelo entrenado
Una vez entrenado, el modelo se guarda en un archivo .pkl mediante joblib.
Esto permite reutilizarlo posteriormente para realizar predicciones sin necesidad de volver a entrenarlo, ahorrando tiempo y recursos.

In [27]:

# Cargamos el archivo .csv 
import pandas as pd
path = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced_preprocessed.csv'

df = pd.read_csv(path)
df.head()

,reviewText,overall
0,"['final', 'fantasy', 'vii', 'ps1', 'consider',...",2
1,"['yeah', 'ok', 'easy', 'use', 'control', 'ol',...",3
2,"['star', 'war', 'force', 'unleash', 'swtfu', '...",3
3,"['review', 'contain', 'minor', 'spoiler', 'wri...",2
4,"['I', 'm', 'have', 'problem', 'get', 'disconne...",1


In [28]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['reviewText'])  # matriz dispersa


In [29]:
print(vectorizer.get_feature_names_out())

['007' '007inspired' '01' ... 'zynga' 'zzz' 'zzzzzp']


In [30]:
import pandas as pd
doc_term_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
doc_term_matrix.head()

,007,007inspired,01,010,0109,010originality,010replayable,010this,01172014rate,03,...,zs,zssamsung,zuma,zumba,zurg,zurgs,zurkon,zynga,zzz,zzzzzp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
y = df['overall']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.75,
    test_size=0.25,
    random_state=42,
    shuffle=True
)
lr_model = LogisticRegression(max_iter=500, solver='lbfgs')
y = df['overall']
lr_model.fit(X_train, y_train)
# guardamos el modelo entrenado


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


In [34]:
# Código completo siguiendo buenas prácticas de ML
# Cargamos el archivo .csv: 
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import uniform, randint
#importar model

#  Parámetros / rutas 
csv_path = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced_preprocessed.csv'


#  Cargar datos 
df = pd.read_csv(csv_path)
df['reviewText'] = df['reviewText'].astype(str)  # asegurar strings
X_text = df['reviewText']
y = df['overall']

# -rain/test split ANTES de cualquier 'fit' del vectorizador 
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text,
    y,
    train_size=0.75,
    test_size=0.25,
    random_state=42,
    shuffle=True,
    stratify=y  # mantiene proporción de clases en ambos sets
)
# Definir un pipeline con CountVectorizer + LogisticRegression
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),             # default params, se optimizarán
    ('clf', LogisticRegression(solver="saga", max_iter=2000, n_jobs=-1))
])

# Espacio de búsqueda (RandomizedSearchCV para velocidad)
param_distributions = {
    'vectorizer__ngram_range': [(1,1), (1,2)],          # 
    'vectorizer__min_df': [1, 2, 3, 5],
    'vectorizer__max_df': [0.7, 0.85, 0.9, 0.95],
    'vectorizer__max_features': [5000, 10000, 20000, None],
    'vectorizer__binary': [False, True],
    'clf__penalty': ['l2', 'elasticnet'],
    'clf__C': uniform(0.01, 10),                        # muestreo continuo para C entre 0.01 y 10
    'clf__l1_ratio': uniform(0, 1)                      
}

# Configurar RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=40,              # número de combinaciones a probar 
    scoring='accuracy',     
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# --- Ajustar el pipeline (fit con X_train solamente) ---
search.fit(X_train_text, y_train)
print("Mejores parámetros encontrados:")
print(search.best_params_)
print("Mejor score CV:", search.best_score_)

# --- Evaluación sobre el set de test usando el mejor estimador ---
best_pipeline = search.best_estimator_
y_pred = best_pipeline.predict(X_test_text)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

#  Guardar artefactos (modelo + vectorizador + test raw)
model_dir = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\modelos\Logistic_regression_bag_of_words0-5'
joblib.dump(model, os.path.join(model_dir, "model.pkl"))
joblib.dump(vectorizer, os.path.join(model_dir, "vectorizer.pkl"))
joblib.dump((X_test_text.reset_index(drop=True), y_test.reset_index(drop=True)),
            os.path.join(model_dir, "test_data_raw.pkl"))

print("Guardado completado en:", model_dir)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Mejores parámetros encontrados:
{'clf__C': 1.205942459383017, 'clf__l1_ratio': 0.713244787222995, 'clf__penalty': 'l2', 'vectorizer__binary': False, 'vectorizer__max_df': 0.85, 'vectorizer__max_features': 20000, 'vectorizer__min_df': 3, 'vectorizer__ngram_range': (1, 2)}
Mejor score CV: 0.3816
Accuracy: 0.42
              precision    recall  f1-score   support

           1       0.53      0.56      0.54       250
           2       0.32      0.30      0.31       250
           3       0.32      0.30      0.31       250
           4       0.40      0.39      0.39       250
           5       0.50      0.56      0.53       250

    accuracy                           0.42      1250
   macro avg       0.41      0.42      0.42      1250
weighted avg       0.41      0.42      0.42      1250

Matriz de confusión:
[[139  58  18  18  17]
 [ 68  74  59  31  18]
 [ 32  56  75  49  38]
 [ 12  26  49  97  66]
 [ 10  20  31  49 140]]
Guardado completado en: C:\Users\rammu\Documents\projects\NLP_04

# Bola Extra: Modelo de deep learning utilizando BERT 

Bert es un modelo ya preentrenado que podemos utilizar para hacer un análisis de sentimiento 
En este caso, utilizaremos 2 distintos modelos que se utilizan en secuencia: 
- En primer lugar, DistilBERT procesa la oración y la pasa al siguiente modelo, creando un word embedding 
- El segundo modelo, un modelo de regresión logística como el que hemos implementado como primer modelo procesará los resultados extraídos por DistilBERT y clasificará la oración como positiva o negativa

In [ ]:
#Instalamos la librería hf_xet, necesaria para la siguiente práctica
!pip install hf_xet

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pandas as pd
#En este caso, dado que Bert incluye su propio Tokenizador, vamos a cargar el dataset sin limpiar
path = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced.csv'

df = pd.read_csv(path)
df.head()

c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3DLWSSO2LVC1E,B000FDOTIQ,Kora,"[4, 6]",Final Fantasy VII for the ps1 was (and still i...,2,More like a movie than a game.,1164844800,"11 30, 2006"
1,AJIDIVBILJKO0,B002DY9KHU,MekoRush,"[0, 0]",Yeah it's ok. it was easy to get use to the co...,3,"Sneakly reviewing this game, it's ok",1334534400,"04 16, 2012"
2,A3PZ4AXTY9J1DZ,B000R39GPA,Jason Ralsky,"[2, 2]",Star Wars: The Force Unleashed (SW:TFU) at lau...,3,Misses the mark of past Jedi games just barely,1287964800,"10 25, 2010"
3,AW3BDUZUFZMFX,B0000A92KZ,Joshua T. Garcia,"[1, 2]",This review may contain minor spoilers. It was...,2,The Fall of Max Payne,1318118400,"10 9, 2011"
4,A1GBZD75UNMD2B,B001TOQ8R0,"Adgear ""Derebu""","[8, 9]",I'm having the same problems as everyone else ...,1,Don't buy this game!!,1269388800,"03 24, 2010"


In [ ]:
# Eliminamos todas la columnas que no nos interesan (todas salvo 'reviewText' y 'overall')
df = df[['reviewText', 'overall']]
df.head()

,reviewText,overall
0,Final Fantasy VII for the ps1 was (and still i...,2
1,Yeah it's ok. it was easy to get use to the co...,3
2,Star Wars: The Force Unleashed (SW:TFU) at lau...,3
3,This review may contain minor spoilers. It was...,2
4,I'm having the same problems as everyone else ...,1


In [ ]:
# Importamos el modelo DistilBERT y su tokenizador
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Los cargamos en la memoria del notebook 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
# Importamos el modelo DistilBERT y su tokenizador
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Los cargamos en la memoria del notebook 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
# inicializamos el tokenizador y el modelo
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# definir dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Tokenizar toda la columna EN BLOQUE pero pidiendo tensores PyTorch.

encodings = tokenizer(
    df["reviewText"].tolist(),
    padding=True,          
    truncation=True,       
    max_length=512,
    return_tensors="pt"    
)
# Esto devuelve tensores ya paddeados al max del batch (sequence length <= 512
#
input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]

# DataLoader para procesar por lotes y evitar OOM
batch_size = 16  
dataset = TensorDataset(input_ids, attention_mask)
loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True)  # pin_memory útil si hay GPU

# Pasar por el modelo por lotes (mover cada batch al dispositivo)
all_embeddings = []
with torch.no_grad():
    for b_input_ids, b_att_mask in loader:
        b_input_ids = b_input_ids.to(device, non_blocking=True)
        b_att_mask = b_att_mask.to(device, non_blocking=True)

        outputs = model(b_input_ids, attention_mask=b_att_mask)
        last_hidden = outputs.last_hidden_state  # (bs, seq_len, hidden_dim)

        # mean pooling considerando attention_mask
        mask = b_att_mask.unsqueeze(-1)
        summed = (last_hidden * mask).sum(1)
        counts = mask.sum(1).clamp(min=1)
        pooled = (summed / counts).cpu()  # mover a CPU para concatenar sin usar VRAM

        all_embeddings.append(pooled)

embeddings = torch.cat(all_embeddings, dim=0)  # (n_samples, hidden_dim)
print(embeddings.shape)

features = embeddings.numpy()
labels = df['overall'].values


c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


torch.Size([5000, 768])


In [ ]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/p

best parameters:  {'C': 15.789557894736841}
best scrores:  0.4453333333333333


c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [ ]:
lr_clf.score(test_features, test_labels)

0.4544

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.205 (+/- 0.00)


In [33]:
# Como parte de las buenas prácticas, vamos a incluir todo el proceso en una sola celda
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
import pandas as pd
#En este caso, dado que Bert incluye su propio Tokenizador, vamos a cargar el dataset sin limpiar
path = r'C:\Users\rammu\Documents\projects\NLP_04\práctica\reviews_Video_Games_5_balanced.csv'

df = pd.read_csv(path)
df.head()
# Eliminamos todas la columnas que no nos interesan (todas salvo 'reviewText' y 'overall')
df = df[['reviewText', 'overall']]
df.head()
# Importamos el modelo DistilBERT y su tokenizador
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Los cargamos en la memoria del notebook 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# Importamos el modelo DistilBERT y su tokenizador
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Los cargamos en la memoria del notebook 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
# inicializamos el tokenizador y el modelo
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# definir dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Tokenizar toda la columna EN BLOQUE pero pidiendo tensores PyTorch.

encodings = tokenizer(
    df["reviewText"].tolist(),
    padding=True,          
    truncation=True,       
    max_length=512,
    return_tensors="pt"    
)
# Esto devuelve tensores ya paddeados al max del batch (sequence length <= 512
#
input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]

# DataLoader para procesar por lotes y evitar OOM
batch_size = 16  
dataset = TensorDataset(input_ids, attention_mask)
loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True)  # pin_memory útil si hay GPU

# Pasar por el modelo por lotes (mover cada batch al dispositivo)
all_embeddings = []
with torch.no_grad():
    for b_input_ids, b_att_mask in loader:
        b_input_ids = b_input_ids.to(device, non_blocking=True)
        b_att_mask = b_att_mask.to(device, non_blocking=True)

        outputs = model(b_input_ids, attention_mask=b_att_mask)
        last_hidden = outputs.last_hidden_state  # (bs, seq_len, hidden_dim)
                # mean pooling considerando attention_mask
        mask = b_att_mask.unsqueeze(-1)
        summed = (last_hidden * mask).sum(1)
        counts = mask.sum(1).clamp(min=1)
        pooled = (summed / counts).cpu()  # mover a CPU para concatenar sin usar VRAM

        all_embeddings.append(pooled)

embeddings = torch.cat(all_embeddings, dim=0)  # (n_samples, hidden_dim)
print(embeddings.shape)

features = embeddings.numpy()
labels = df['overall'].values
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

from sklearn.model_selection import GridSearchCV

parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)
# 
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


torch.Size([5000, 768])


c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/p

best parameters:  {'C': 5.263252631578947}
best scrores:  0.4549333333333333
Dummy classifier score: 0.206 (+/- 0.00)


c:\Users\rammu\Documents\projects\NLP_04\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
